In [2]:
import os
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import multiprocessing
import gzip
import pandas as pd
import csv
import string
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict

In [3]:
listing_titles = pd.read_csv('./dataset/Listing_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

In [4]:
corpus = listing_titles['Title'].to_list()
corpus = [item.translate(str.maketrans('', '', string.punctuation)) for item in corpus]
model_name = "word_vecs"

In [5]:
print (len(corpus))


phrases = Phrases(corpus, min_count=30, progress_per=10000)
print ("Made Phrases")

19999921
Made Phrases


In [6]:
bigram = Phraser(phrases)
print ("Made Bigrams")

Made Bigrams


In [7]:
sentences = phrases[corpus]
print ("Found sentences")
word_freq = defaultdict(int)

for sent in sentences:
    for i in sent:
        word_freq[i]+=1

print (len(word_freq))


Found sentences
2499


In [8]:
print ("Training model now...")
w2v_model = Word2Vec(min_count=20,
                    window=2.1,
                    sample=6e-5,
                    alpha=0.03,
                    min_alpha=0.0007,
                    negative=20,
                    workers=8
                    )
w2v_model.build_vocab(sentences, progress_per=10000)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
os.makedirs("./trainset", exist_ok=True)
w2v_model.wv.save_word2vec_format(f"trainset/{model_name}.txt")

Training model now...


: 

In [ ]:
!./train.sh